In [ ]:
#Importing all the necessary libraries
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
nltk.download("stopwords")
import re
import warnings
warnings.filterwarnings("ignore")
import os
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Extracting all the files.
all_files=os.listdir('/content/drive/MyDrive/IR assignment/Humor,Hist,Media,Food')

In [ ]:
# Creating a dictionary which contains all file names and their respective contents.
file_content_dict={'filename':[],'content':[]}
for i in all_files:
  file1 = open("/content/drive/MyDrive/IR assignment/Humor,Hist,Media,Food/"+i,"r",encoding='utf-8',errors='ignore')
  file_content_dict['filename'].append(i)
 # text = file1.read().decode(errors='replace')
  text=file1.read()
  file_content_dict['content'].append(text)

In [ ]:
# Creating a dataframe which consists two columns i.e., filename and content
file_content=pd.DataFrame(file_content_dict)

In [ ]:
file_content

,filename,content
0,rinaldos.txt,\n Rinaldo's Laws\n ...
1,bbq.txt,\n \n AGREEMENT FOR PARTICIPA...
2,mothers.txt,\n* MOTHER'S DICTIONARY *\n\nAMNESIA: conditio...
3,harmful.hum,DIRTY AIR\n\n Scientists are now saying that ...
4,mowers.txt,\n\tDepartment of Agriculture Bulletin #265\n\...
...,...,...
1128,mensroom.jok,From: dwallach@ultra.com (Dan Wallach)\nNewsgr...
1129,female.jok,From nobody@prles2.UUCP Sun Apr 9 15:32:48 19...
1130,goforth.hum,\cGO FORTH AND WAIT: A play in one scene\n ...
1131,grommet.hum,HELP! THE GROMMET'S MISSING!\n\n I had a happ...


# **Preprocessing**

In [ ]:
# Converted all the words in lower case.
# Removed all the special characters and punctuation marks.
# Removed all the stopwords from the corpus using NLTK stopwords.
# Performed tokenization on the document corpus.

w_token = nltk.tokenize.WhitespaceTokenizer()
file_content['content']=file_content['content'].str.lower()
stop_word = stopwords.words('english')
file_content['content'] = file_content['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_word)]))
file_content['content'] = file_content['content'].str.replace('[^a-zA-Z0-9]',' ')
file_content['content_tokens']=file_content['content'].apply(w_token.tokenize)

In [ ]:
#Creating unique tokens list and token frequency dictionary.
token_frequency={}
# token_frequency stores the frequency of tokens of the document corpus.
unique_tokens=[]
# unique_tokens stores the list of unique tokes of the document corpus. 
for tokens in file_content['content_tokens']:
  for word in tokens:
          if(word not in token_frequency.keys()):
              token_frequency[word] = {}
              unique_tokens.append(word)
         
              

In [ ]:
# Creating position index list
for i in range(len(all_files)):
    term=file_content.iloc[i,2]
    for position, word in enumerate(term):
        if i in token_frequency[word].keys():
            token_frequency[word][i].append(position)
        else:
            token_frequency[word][i] = [position]

# **Processing the input query**

In [ ]:
# This function takes the query from user and perform all the pre-processing on the query and finally returns the pre-processed query.
def EnterQuery():
  query=input('Enter your query')
  query=query.lower()
  query = re.sub('[^a-zA-Z0-9]', ' ', query)
  stop_word = stopwords.words('english')
  query=query.split()
  query_token=[word for word in query if not word in stop_word]
  return query_token

In [ ]:
def process_query(arg,query_token):
  # The number of tokens = 0: No document retrieved.
  if(arg==0):
    result=[]
    print("Number of documents retrieved are: ",len(result))
    print("The list of document names retrieved are: ",result)  
  
  # The number of tokens = 1: Simply return the documents containing the token.
  elif(arg==1):
    if(query_token[0] in unique_tokens):
        result = set(token_frequency[query_token[0]].keys())
    print("Number of documents retrieved are: ",len(result))
    document_names=[]
    for i in result:
      document_names.append(file_content.iloc[i,0])
    print("The list of document names retrieved are: ",document_names) 

  # The number of tokens = 2: We check the following condition on all the documents: 
  #  Position of token2 - Position of token 1 ==  1
  elif(arg==2):
    result=[]
    temp1 = set(token_frequency[query_token[0]].keys())
    temp2 = set(token_frequency[query_token[1]].keys())
    temp  = list(temp2.intersection(temp1))
    for a in temp:
      for b in token_frequency[query_token[0]][a]:
        for c in token_frequency[query_token[1]][a]:
            if(c-b==1):
                result.append(a)        
    result= list(set(result))
    print("Number of documents retrieved are: ",len(result))
    document_names=[]
    for i in result:
      document_names.append(file_content.iloc[i,0])
    print("The list of document names retrieved are: ",document_names)  
  
#   The number of tokens = 3: We check the following condition on all the documents:
#   Position of token3 - Position of token 2 ==  1 and Position of token2 - Position of token 1 ==  1 and Position of token 3 - Position of token 1 ==2.
  elif(arg==3):
    result=[]
    temp1 = set(token_frequency[query_token[0]].keys())
    temp2 = set(token_frequency[query_token[1]].keys())
    temp3 = set(token_frequency[query_token[2]].keys())
    temp4  = temp2.intersection(temp1)
    temp = list(temp4.intersection(temp3))
    for a in temp:
      for b in token_frequency[query_token[0]][a]:
        for c in token_frequency[query_token[1]][a]:
          for d in token_frequency[query_token[2]][a]:
            if(c-b==1 and d-c==1 and d-b==2):
                result.append(a)        
    result= list(set(result))
    print("Number of documents retrieved are: ",len(result))
    document_names=[]
    for i in result:
      document_names.append(file_content.iloc[i,0])
    print("The list of document names retrieved are: ",document_names)  

#   The number of tokens = 4: We check the following condition on all the documents:
#   Position of token 4 - Position of token 3 ==  1 and Position of token 3 - Position of token 2 ==  1 
#   and Position of token 2 - Position of token 1 ==1 and position of token 4 - Position of token 1 == 3.
  elif(arg==4):
    result=[]
    temp1 = set(token_frequency[query_token[0]].keys())
    temp2 = set(token_frequency[query_token[1]].keys())
    temp3 = set(token_frequency[query_token[2]].keys())
    temp5 = set(token_frequency[query_token[3]].keys())
    temp4 = temp2.intersection(temp1)
    temp6 = temp3.intersection(temp5)
    temp = list(temp4.intersection(temp6))
    for a in temp:
      for b in token_frequency[query_token[0]][a]:
        for c in token_frequency[query_token[1]][a]:
          for d in token_frequency[query_token[2]][a]:
            for e in token_frequency[query_token[3]][a]:
              if(c-b==1 and d-c==1 and d-b==2 and e-d==1 and e-c==2 and e-b==3):
                  result.append(a)        
    result= list(set(result))
    print("Number of documents retrieved are: ",len(result))
    document_names=[]
    for i in result:
      document_names.append(file_content.iloc[i,0])
    print("The list of document names retrieved are: ",document_names) 

#     The number of tokens = 5: We check the following condition on all the documents:
#     Position of token 5 - Position of token 4 ==1 and Position of token 4 - Position of token 3 ==  1 
#     and Position of token 3 - Position of token 2 ==  1 and Position of token 2 - Position of token 1 ==1 and position of token 5 - Position of token 1 == 4.
  
  elif(arg==5):
    result=[]
    temp1 = set(token_frequency[query_token[0]].keys())
    temp2 = set(token_frequency[query_token[1]].keys())
    temp3 = set(token_frequency[query_token[2]].keys())
    temp5 = set(token_frequency[query_token[3]].keys())
    temp7 = set(token_frequency[query_token[4]].keys())
    temp4 = temp2.intersection(temp1)
    temp6 = temp3.intersection(temp5)
    temp8=  temp4.intersection(temp6)
    temp = list(temp8.intersection(temp7))
    for a in temp:
      for b in token_frequency[query_token[0]][a]:
        for c in token_frequency[query_token[1]][a]:
          for d in token_frequency[query_token[2]][a]:
            for e in token_frequency[query_token[3]][a]:
              for f in token_frequency[query_token[4]][a]:
                if(c-b==1 and d-c==1 and d-b==2 and e-d==1 and e-c==2 and e-b==3 and f-e==1 and f-b==4):
                    result.append(a)        
    result= list(set(result))
    print("Number of documents retrieved are: ",len(result))
    document_names=[]
    for i in result:
      document_names.append(file_content.iloc[i,0])
    print("The list of document names retrieved are: ",document_names) 
  else:
    print('We have assumed that the query length is less than or equal to 5.\nPlease Try again!!!')

In [ ]:
query=EnterQuery()
print("The tokens generated from query are: ",query)
process_query(len(query),query)

Enter your queryWoods World Championship Chili
The tokens generated from query are:  ['woods', 'world', 'championship', 'chili']
Number of documents retrieved are:  2
The list of document names retrieved are:  ['woods.txt', 'chili.txt']
